In [6]:
# notebook dependencies
%matplotlib inline
import matplotlib as mlp
mlp.rcParams['figure.dpi'] = 300

from skimpy import clean_columns
import pandas as pd
import numpy as np

# visualization libraries/modules
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook_connected"
sns.set()
 

import env
from env import user, password, host, get_db_url

In [ ]:
query = '''
        SELECT date,
                time,
                path as endpoint,
                user_id,
                cohort_id,
                ip as source_ip
        FROM logs
        '''
# creating the first url
url = get_db_url( "curriculum_logs", user,  host, password)

df_logs = pd.read_sql(query, url)
df_logs.head()

In [ ]:

# creating the cohorts query 
query = '''
        SELECT id as cohort_id,
                name as cohort_name
        FROM cohorts
        '''

# creating the url
url = get_db_url("curriculum_logs", user, host, password)

df_cohorts = pd.read_sql(query, url)
df_cohorts.head()

In [ ]:
# merging the two (2) tables together 

df = pd.merge(
    df_logs, 
    df_cohorts,
    on = "cohort_id"
)

print(f'merged df shape: {df.shape}')
df.head()

In [ ]:
df.info()

In [ ]:
# converting date column to proper pd.datetime 

df["datetime"] = df["date"] + " " + df["time"]
df = df.drop(columns = ["date", "time"])

df["datetime"] = pd.to_datetime(df["datetime"])
df.head()

In [ ]:
# setting the date column to index

df = df.set_index("datetime").rename_axis(None)
df.head()

In [7]:
def get_curriculum_logs():

    # creating the corriculum logs url
    url = get_db_url("curriculum_logs", user, host, password)

    # creating the logs query
    logs_query = '''
                SELECT date,
                time,
                path as endpoint,
                user_id,
                cohort_id,
                ip as source_ip
                FROM logs'''

    # creating the logs df
    df_logs = pd.read_sql(logs_query, url)

    # creating the cohorts query 
    cohort_query = '''
                SELECT id as cohort_id,
                start_date as cohort_start,
                end_date as cohort_end,
                name as cohort_name
                FROM cohorts'''

    # creating the cohorts df
    df_cohorts = pd.read_sql(cohort_query, url)

    # merging the two dataframes
    df = pd.merge(
                df_logs, 
                df_cohorts,
                on = "cohort_id")

    # setting user/cohort id to proper data type
    df[["user_id", "cohort_id"]] = df[["user_id", "cohort_id"]].astype(object)

    # combining date and time & dropping previous columns
    df["datetime"] = df["date"] + " " + df["time"]
    df = df.drop(columns = ["date", "time"])

    # converting datetime column to proper pd.datetime 
    df["datetime"] = pd.to_datetime(df["datetime"])

    # setting the date column to index
    df = df.set_index("datetime").rename_axis(None).sort_index()
    
    # creating a day column 
    df["day"] = df.index.strftime("%A")

    # creating a month column 
    df["month"] = df.index.strftime("%B")

    # cleaning remaining column names
    df = clean_columns(df)

    # print the new merged df shape
    print(f'df shape: {df.shape}')

    return df

In [8]:
# let's try the function

df = get_curriculum_logs()
df.head()

0 column names have been cleaned

df shape: (847330, 9)


,endpoint,user_id,cohort_id,source_ip,cohort_start,cohort_end,cohort_name,day,month
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,2015-09-22,2016-02-06,Hampton,Friday,January
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,2015-09-22,2016-02-06,Hampton,Friday,January
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,2015-09-22,2016-02-06,Hampton,Friday,January
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,2015-09-22,2016-02-06,Hampton,Friday,January
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,2018-01-08,2018-05-17,Teddy,Friday,January


In [9]:
pd.DataFrame(df.groupby([df.index, 'user_id', 'cohort_name'])['endpoint'].agg('count'))

,,,endpoint
,user_id,cohort_name,
2018-01-26 09:55:03,1,Hampton,1
2018-01-26 09:56:02,1,Hampton,1
2018-01-26 09:56:05,1,Hampton,1
2018-01-26 09:56:06,1,Hampton,1
2018-01-26 09:56:24,2,Teddy,1
...,...,...,...
2021-04-21 16:41:51,875,Marco,1
2021-04-21 16:42:02,64,Staff,1
2021-04-21 16:42:09,64,Staff,1


In [10]:
# Is it possible to identify the user_id of a staff member?

cohort_lst = df["cohort_name"].sort_values().unique()
cohort_lst

array(['Andromeda', 'Apex', 'Apollo', 'Arches', 'Badlands', 'Bash',
       'Bayes', 'Betelgeuse', 'Ceres', 'Curie', 'Darden', 'Deimos',
       'Denali', 'Easley', 'Europa', 'Everglades', 'Florence', 'Fortuna',
       'Franklin', 'Ganymede', 'Glacier', 'Hampton', 'Hyperion', 'Ike',
       'Joshua', 'Jupiter', 'Kalypso', 'Kings', 'Lassen', 'Luna',
       'Mammoth', 'Marco', 'Neptune', 'Niagara', 'Oberon', 'Olympic',
       'Pinnacles', 'Quincy', 'Sequoia', 'Staff', 'Teddy', 'Ulysses',
       'Voyageurs', 'Wrangell', 'Xanadu', 'Yosemite', 'Zion'],
      dtype=object)

### Exercise: Identify students who present anomalous activity using the Bollinger Band method, but reduce K to 2

In [ ]:
# creating the resampling by day

daily_pages = pd.DataFrame(df['endpoint'].resample("D").count()).rename(columns = {'endpoint': 'pages_visited'})
daily_pages